Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Analyze categorical binary classification model predictions
_**This notebook showcases how to use error analysis to help understand the errors in a binary classification model.**_


## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Project](#Project)
1. [Run model explainer locally at training time](#Explain)
    1. Train a binary classification model
    1. Explain the model
        1. Generate global explanations
        1. Generate local explanations
1. [Visualize results](#Analyze)
1. [Next steps](#Next)

<a id='Introduction'></a>
## 1. Introduction

This notebook illustrates how to locally use interpret-community to help interpret binary classification model predictions at training time.  It demonstrates the API calls needed to obtain the global and local interpretations along with an interactive visualization dashboard for discovering patterns in data and explanations.

Three tabular data explainers are demonstrated: 
- TabularExplainer (SHAP)
- MimicExplainer (global surrogate)
- PFIExplainer.

| ![Interpretability Toolkit Architecture](./img/interpretability-architecture.png) |
|:--:|
| *Interpretability Toolkit Architecture* |

<a id='Project'></a>       
## 2. Project

The goal of this project is to classify breast cancer diagnosis with scikit-learn and locally running the model explainer:

1. Train a SVM classification model using Scikit-learn
2. Run 'explain_model' globally and locally with full dataset in local mode, which doesn't contact any Azure services.
3. Visualize the global and local explanations with the visualization dashboard.

<a id='Setup'></a>
## 3. Setup

If you are using Jupyter notebooks, the extensions should be installed automatically with the package.
If you are using Jupyter Labs run the following command:
```
(myenv) $ jupyter labextension install @jupyter-widgets/jupyterlab-manager
```


<a id='Explain'></a>
## 4. Run model explainer locally at training time

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn import svm
import pandas as pd
import zipfile
from lightgbm import LGBMClassifier

# Explainers:
# 1. SHAP Tabular Explainer
#from interpret.ext.blackbox import TabularExplainer
from interpret.ext.blackbox import TabularExplainer

# OR

# 2. Mimic Explainer
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LinearExplainableModel
from interpret.ext.glassbox import LGBMExplainableModel

# You can use one of the following four interpretable models as a global surrogate to the black box model
#from interpret.ext.glassbox import LGBMExplainableModel
#from interpret.ext.glassbox import LinearExplainableModel
#from interpret.ext.glassbox import SGDExplainableModel
#from interpret.ext.glassbox import DecisionTreeExplainableModel

# OR

# 3. PFI Explainer
#from interpret.ext.blackbox import PFIExplainer 

### Load the UCI adult census income dataset

In [ ]:
outdirname = 'erroranalysis.12.3.20'
try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve
zipfilename = outdirname + '.zip'
urlretrieve('https://publictestdatasets.blob.core.windows.net/data/' + zipfilename, zipfilename)
with zipfile.ZipFile(zipfilename, 'r') as unzip:
    unzip.extractall('.')

In [ ]:
train_data = pd.read_csv('adult-train.csv')
test_data = pd.read_csv('adult-test.csv')

In [ ]:
from sklearn.model_selection import train_test_split
# test_data, _ = train_test_split(test_data, test_size=0.997, random_state=7)
test_data, _ = train_test_split(test_data, test_size=0.9, random_state=7)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def split_label(dataset):
    X = dataset.drop(['income'], axis=1)
    y = dataset[['income']]
    return X, y

def clean_data(X, y):
    features = X.columns.values.tolist()
    classes = y['income'].unique().tolist()
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = feat_pipe.fit_transform(X)
    return X, feat_pipe, features, classes

X_train_original, y_train = split_label(train_data)
X_train, feat_pipe, features, classes = clean_data(X_train_original, y_train)
X_test_original, y_test = split_label(test_data)
y_test = y_test['income'].to_numpy()
X_test = feat_pipe.transform(X_test_original)
features = train_data.columns.values[1:].tolist()
classes = y_train['income'].unique().tolist()
categorical_features = train_data.dtypes[train_data.dtypes == 'object'].index.values[1:].tolist()

### Train a LightGBM classification model, which you want to analyze

In [ ]:
clf = LGBMClassifier(n_estimators=1)
model = clf.fit(X_train, y_train['income'])

### Explain predictions on your local machine

In [ ]:
from interpret_community.common.constants import ShapValuesOutput, ModelTask
# 1. Using SHAP TabularExplainer
model_task = ModelTask.Classification
explainer = MimicExplainer(model, X_train_original, LGBMExplainableModel,
                            augment_data=True, max_num_of_augmentations=10,
                            features=features, classes=classes, model_task=model_task,
                            transformations=feat_pipe)




# 2. Using MimicExplainer
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns. 
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
# explainer = MimicExplainer(model, 
#                            x_train, 
#                            LGBMExplainableModel, 
#                            augment_data=True, 
#                            max_num_of_augmentations=10, 
#                            features=breast_cancer_data.feature_names, 
#                            classes=classes)





# 3. Using PFIExplainer

# Use the parameter "metric" to pass a metric name or function to evaluate the permutation. 
# Note that if a metric function is provided a higher value must be better.
# Otherwise, take the negative of the function or set the parameter "is_error_metric" to True.
# Default metrics: 
# F1 Score for binary classification, F1 Score with micro average for multiclass classification and
# Mean absolute error for regression

# explainer = PFIExplainer(model, 
#                          features=breast_cancer_data.feature_names, 
#                          classes=classes)

### Generate global explanations
Explain overall model predictions (global explanation)

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(X_test_original)

# Note: if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_test, true_labels=y_test)

In [ ]:
# Sorted SHAP values
print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
# Corresponding feature names
print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
# Feature ranks (based on original order of features)
print('global importance rank: {}'.format(global_explanation.global_importance_rank))

# Note: Do not run this cell if using PFIExplainer, it does not support per class explanations
# Per class feature names
print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))
# Per class feature importance values
print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

In [ ]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

In [ ]:
from sklearn.pipeline import Pipeline
dashboard_pipeline = Pipeline(steps=[('preprocess', feat_pipe), ('model', model)])

<a id='Analyze'></a>
## 5. Analyze
Analyze using Error Analysis Dashboard

In [ ]:
from raiwidgets import ErrorAnalysisDashboard

In [ ]:
ErrorAnalysisDashboard(global_explanation, dashboard_pipeline, dataset=X_test_original,
                       true_y=y_test, categorical_features=categorical_features)

<a id='Next'></a>
## 6. Next steps
Learn about other use cases of the explain package on a:
       
1. [Training time: regression problem](./explain-regression-local.ipynb)
1. [Training time: multiclass classification problem](./explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](./simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](./advanced-feature-transformations-explain-local.ipynb)